In [35]:
import numpy as np
import time as T
from PIL import Image
FBIT = 13
VBIT = 1 << FBIT
HBIT = 1 << (FBIT - 1)
CR = lambda x,y: ((x << FBIT)//y)

## Resize for nearest neighbor

In [19]:
W, H, C = 400, 300, 3
im = np.arange( W * H * C, dtype=np.uint8).reshape(C, H, W).transpose(1, 2, 0)
rW, rH, rC = 123, 200, 3

In [32]:
def CLIP(val, lower, upper):
    val = lower if val < lower else val
    val = upper if val > upper else val
    return val

In [21]:
def resizeNN(im, w, h):
    ih, iw, ic = im.shape
    w_r = iw/w; h_r = ih/h
    img = np.empty((h, w, ic), dtype=np.uint8)
    p =0
    for i in range(h):
        hi = int(i*h_r + 0.5)
        for j in range(w):
            wi = int(j*w_r + 0.5)
            for k in range(ic):
                img[i,j,k] = im[hi, wi, k]
    return img

def resizeANN(im, w, h):
    ih, iw, ic = im.shape
    w_r = CR(iw,w); h_r = CR(ih,h)
    img = np.empty((h, w, ic), dtype=np.uint8)
    p =0
    for i in range(h):
        hi = (i * h_r + HBIT) >> FBIT
        for j in range(w):
            wi = (j * w_r + HBIT) >> FBIT 
            for k in range(ic):
                img[i,j,k] = im[hi, wi, k]
    return img



In [23]:
dog = Image.open("Dog.jpg")
dog.show()

In [24]:
rdog = resizeANN(np.array(dog), rH, rW)
im = Image.fromarray(rdog)
im.show()

In [25]:
## Time benchmark for floating point calc and fixed point calc

In [26]:
import time as T

RUN = 50
W,H = 224, 224
ti = T.time()
for i in range(RUN):
    rdog = resizeNN(np.array(dog), W, H)
toc1 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = resizeANN(np.array(dog), W, H)
toc2 = T.time() - ti
print("toc1 vs toc2", toc1, toc2)

toc1 vs toc2 4.197096347808838 3.9506547451019287


In [29]:
import time as T

RUN = 10
W,H = 224, 224
ti = T.time()
for i in range(RUN):
    rdog = resizeANN(np.array(dog), W, H)
toc1 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = resizeNN(np.array(dog), W, H)
toc2 = T.time() - ti
print("toc1 vs toc2", toc1, toc2)

toc1 vs toc2 0.841686487197876 0.7871720790863037


1

In [ ]:
"""
https://stackoverflow.com/questions/12729228/simple-efficient-bilinear-interpolation-of-images-in-numpy-and-python
"""

In [78]:
def resizeBL(im, w, h):
    im_h, im_w, im_c = im.shape
    h_r = im_h / h
    w_r = im_w / w
    img = np.empty((h,w,im_c))
    
    for j in range(h):
        h0 = j * h_r
        n_h0 = int(h0)
        n_h1 = n_h0 + 1
        n_h1 = np.clip(n_h1, 0, im_h-1)
        h0 = h0 % 1
        h0V = 1-h0
        for k in range(w):
            w0 = k * w_r
            n_w0 = int(w0)
            n_w1 = n_w0 + 1
            n_w1 = np.clip(n_w1, 0, im_w-1)
            w0 = w0 % 1
            w0V = 1 - w0
            for i in range(im_c):
                #pix00 = im[n_h0, n_w0, i]
                #pix01 = im[n_h0, n_w1, i]
                #pix10 = im[n_h1, n_w0, i]
                #pix11 = im[n_h1, n_w1, i]
                #print("w0:%.2f, h0:%.2f, n_h0:%d, n_w0:%d, h_r:%.2f, w_r:%.2f" % (w0, h0, n_h0, n_w0, h_r, w_r))
                row_pix0 = im[n_h0, n_w0, i] * w0 + im[n_h0, n_w1, i] * w0V               
                row_pix1 = im[n_h1, n_w0, i] * w0 + im[n_h1, n_w1, i] * w0V

                img[j, k, i] = int(row_pix0 * h0 + row_pix1 * h0V)
                img[j, k, i] = CLIP(img[j, k, i], 0 ,255)

    return img.astype(np.uint8)             

def resizeABL(im, w, h):
    im_h, im_w, im_c = im.shape
    h_r = CR(im_h, h)
    w_r = CR(im_w, w)
    img = np.empty((h,w,im_c))

    for j in range(h):
        h0 = j * h_r
        n_h0 = h0 >> FBIT 
        n_h1 = n_h0 + 1
        #n_h1 = CLIP(n_h1, 0, im_h-1)
        h0 = (h0 + HBIT) % VBIT
        h0V = VBIT-h0
        for k in range(w):
            w0 = k * w_r
            n_w0 = w0 >> FBIT 
            n_w1 = n_w0 + 1
            #n_w1 = CLIP(n_w1, 0, im_w-1)
            w0 = (w0 + HBIT) % VBIT
            w0V = VBIT-w0
            for i in range(im_c):
                #pix00 = im[n_h0, n_w0, i]
                #pix01 = im[n_h0, n_w1, i]
                #pix10 = im[n_h1, n_w0, i]
                #pix11 = im[n_h1, n_w1, i]
                row_pix0 = ((im[n_h0, n_w0, i] * w0 + im[n_h0, n_w1, i] * w0V) + HBIT) >> FBIT
                row_pix1 = ((im[n_h1, n_w0, i] * w0 + im[n_h1, n_w1, i] * w0V) + HBIT) >> FBIT

                img[j, k, i] = ((row_pix0 * h0 + row_pix1 * h0V) + HBIT) >> FBIT
                #img[j, k, i] = CLIP(img[j, k, i], 0, 255)
    return img.astype(np.uint8)             

In [79]:
dog = Image.open("Dog.jpg")
dog_array = np.array(dog)


In [80]:
dog_bi = resizeBL(dog_array, 200, 144)

In [81]:
ndog = Image.fromarray(dog_bi)
ndog.show()

In [82]:
dog_bi = resizeABL(dog_array, 200, 144)

In [83]:
ndog = Image.fromarray(dog_bi)
ndog.show()

In [84]:
import time as T

RUN = 10
W,H = 224, 224
ti = T.time()
for i in range(RUN):
    rdog = resizeBL(np.array(dog), W, H)
toc1 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = resizeABL(np.array(dog), W, H)
toc2 = T.time() - ti
print("toc1 vs toc2", toc1, toc2)

toc1 vs toc2 21.73677086830139 16.525208473205566


In [50]:
import cv2

In [73]:
RUN = 100
W,H = 224, 224
ti = T.time()
for i in range(RUN):
    rdog = cv2.resize(np.array(dog), (W, H), interpolation=cv2.INTER_NEAREST)
toc1 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = cv2.resize(np.array(dog), (W, H), interpolation=cv2.INTER_LINEAR)
toc2 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = cv2.resize(np.array(dog), (W, H), interpolation=cv2.INTER_AREA)
toc3 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = cv2.resize(np.array(dog), (W, H), interpolation=cv2.INTER_CUBIC)
toc4 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = cv2.resize(np.array(dog), (W, H), interpolation=cv2.INTER_LANCZOS4)
toc5 = T.time() - ti
print("toc1:%.4f\ntoc2:%.4f\ntoc3:%.4f\ntoc4:%.4f\ntoc5:%f" %( toc1, toc2, toc3, toc4, toc5))

toc1:0.3082
toc2:0.3189
toc3:0.6765
toc4:0.3475
toc5:0.517593
